## This file get info already stored in chroma database (the best version)

### Import the libraries to get the env variables

In [52]:
import os
from dotenv import load_dotenv

In [53]:
load_dotenv()

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"

In [54]:
OPENAI_API_KEY 

'9ac347d13e834f288a2076ff9c7b418a'

In [68]:
# database = "./conventia_viena-database"
database = "C:/Users/Alex/OneDrive/Documente/PersonalRepos/AI/LangChain/AzureCognitiveSearch/barack_obama-database"
query = "After the family moved to California, where was Stanley enrolled?"

### Import the libraries to connect to chroma database

In [56]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from chromadb.utils import embedding_functions


In [61]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


##### Connect to chroma database

In [71]:
db3 = Chroma(persist_directory=database, embedding_function=embedding_function)
db3

##### Create the retriever and get relevant documents

In [73]:
retriever = db3.as_retriever(search_type="similarity", search_kwargs = {"k": 5})
docs = retriever.get_relevant_documents("After the family moved to California, where was Stanley enrolled?")

len(docs)
print(docs)


[]


#### Import the model 3.5 turbo

In [44]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

llm = AzureChatOpenAI(
    deployment_name = "gpt35-uif54579",
    model_name = "gpt-35-turbo",
    temperature = 0.5
)

In [45]:
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.5, model_kwargs={}, openai_api_key='9ac347d13e834f288a2076ff9c7b418a', openai_api_base='https://sbzdfopenai.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='gpt35-uif54579', openai_api_type='azure', openai_api_version='2023-03-15-preview')

In [46]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever = retriever, return_source_documents = True)


In [47]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print("\n\nSources:")
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [48]:
llm_response = chain(query)
llm_response

{'query': "They too, he said, had big dreams for their daughter. He said his parents met while \nstudying at the University of Hawaii and that they shared not only an improbable love, but also an \nabiding faith in the possibilities of this nation. Barack said that this country’s pride is based on a simple \npremise, summed up in the Declaration of Independence, as 'the true genius of America, a faith in the \nsimple dreams of its people, the insistence on small miracles'",
 'result': "What is the premise that Barack Obama says this country's pride is based on? \n\nBarack Obama says that this country's pride is based on a simple premise, summed up in the Declaration of Independence, as 'the true genius of America, a faith in the simple dreams of its people, the insistence on small miracles'.",
 'source_documents': []}

In [49]:
process_llm_response(llm_response)

What is the premise that Barack Obama says this country's pride is based on? 

Barack Obama says that this country's pride is based on a simple premise, summed up in the Declaration of Independence, as 'the true genius of America, a faith in the simple dreams of its people, the insistence on small miracles'.


Sources:


In [50]:
llm_response['result']

"What is the premise that Barack Obama says this country's pride is based on? \n\nBarack Obama says that this country's pride is based on a simple premise, summed up in the Declaration of Independence, as 'the true genius of America, a faith in the simple dreams of its people, the insistence on small miracles'."